In [1]:
!pip install transformers[torch] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.6 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertLMHeadModel
from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments, TextDataset

import torch
from torch.utils.data import TensorDataset
import pandas as pd

import json
import csv

In [ ]:
#ensuring that each pattern and response is separated
def convert_to_pairs(json_file_path, new_json_file_path):
      with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    new_data = {"intents": []}
    for intent in data["intents"]:
        pairs = [{"pattern": pattern, "response": response} for pattern, response in zip(intent["patterns"], intent["responses"])]
        new_intent = {"tag": intent["tag"], "pairs": pairs}
        new_data["intents"].append(new_intent)

    with open(new_json_file_path, 'w') as new_json_file:
        json.dump(new_data, new_json_file, indent=4)

# Example usage
json_file_path = '/content/PPD_intents.json'
new_json_file_path = 'new_data.json'
convert_to_pairs(json_file_path, new_json_file_path)


In [ ]:
#convert the json data to csv format
def json_to_csv(json_file_path, csv_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    with open(csv_file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["tag", "pattern", "response"])  # Write header

        for intent in data["intents"]:
            tag = intent["tag"]
            for pair in intent["pairs"]:
                pattern = pair["pattern"]
                response = pair["response"]
                writer.writerow([tag, pattern, response])

# Example usage
json_file_path = '/content/new_data.json'
csv_file_path = 'intents.csv'
json_to_csv(json_file_path, csv_file_path)


In [5]:
df = pd.read_csv('intents.csv')

In [6]:
#dropping the unecessary column
df = df.drop(columns=['tag'])

In [7]:
df=df.dropna()

In [ ]:
df

In [9]:
df['pattern']= df['pattern'].str.replace('[^\w\s]','',regex=True) #remove punctuation
df['pattern']= df['pattern'].str.replace('\s+',' ',regex=True) #remove unecessary spaces
df['pattern']= df['pattern'].str.lower() #convert entire string to lowercase

df['response']= df['response'].str.replace('[^\w\s]','',regex=True) #remove punctuation
df['response']= df['response'].str.replace('\s+',' ',regex=True) #remove unecessary spaces
df['response']= df['response'].str.lower() #convert entire string to lowercase


In [10]:
#split into training and validation sets
train_ratio = 0.8
split_index = int(train_ratio * len(df))
train_data = df[:split_index]
val_data = df[split_index:]

In [11]:
print(f"Train size: {len(train_data)}")
print(f"Validation size: {len(val_data)}")


Train size: 140
Validation size: 36


In [12]:
# Instantiate the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
# Tokenize the patterns and responses
max_length = tokenizer.model_max_length
# Tokenize the patterns and responses
train_encodings = tokenizer(list(train_data['pattern']), truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(list(val_data['pattern']), truncation=True, padding=True, return_tensors='pt')

# Tokenize responses and directly return PyTorch tensors
train_y = tokenizer(list(train_data['response']), truncation=True, padding=True, return_tensors='pt')
val_y = tokenizer(list(val_data['response']), truncation=True, padding=True, return_tensors='pt')



In [14]:
# Define PyTorch datasets
class PatternResponseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, y):
        self.encodings = encodings
        self.y = y

    def __getitem__(self, idx):
        item = {key: tensor[idx] for key, tensor in self.encodings.items()}
        item['labels'] = self.y['input_ids'][idx]  # Add labels for MLM
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [15]:
# Create datasets
train_dataset = PatternResponseDataset(train_encodings, train_y)
val_dataset = PatternResponseDataset(val_encodings, val_y)

In [18]:
from transformers import BertTokenizer, BertLMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained and model
model = BertLMHeadModel.from_pretrained('bert-base-uncased')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=50,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    evaluation_strategy='epoch',      #use epochs to evaluate
    save_steps=1000,                  # save checkpoint every n updates steps
    save_total_limit=2,              # limit the total amount of checkpoints
    logging_dir='./logs',            # directory for storing logs
)

# Create data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

# Instantiate Trainer
trainer = Trainer(
    model=model,                         # the instantiated model to be trained
    args=training_args,                  # training arguments
    data_collator=data_collator,         # data collator
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [19]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,6.622776
2,No log,6.203432
3,No log,5.826942
4,No log,5.145572
5,No log,5.678541
6,No log,4.990282
7,No log,5.079800
8,No log,4.322496
9,No log,5.096661
10,No log,5.915179


TrainOutput(global_step=900, training_loss=1.9268799506293404, metrics={'train_runtime': 83.7526, 'train_samples_per_second': 83.58, 'train_steps_per_second': 10.746, 'total_flos': 79167072600000.0, 'train_loss': 1.9268799506293404, 'epoch': 50.0})

In [20]:
#saving the tokenizer and model
save_path = 'my_trained_model/'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('my_trained_model/tokenizer_config.json',
 'my_trained_model/special_tokens_map.json',
 'my_trained_model/vocab.txt',
 'my_trained_model/added_tokens.json')

Inference

In [21]:
from transformers import BertTokenizer, BertLMHeadModel
import re

In [54]:
def generate_text(model, tokenizer, sequence, max_length):
    model = model
    tokenizer = tokenizer
    input= sequence.lower()
    ids = tokenizer.encode(input, return_tensors='pt').to(device)
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        num_beams=3,
        no_repeat_ngram_size=1,
        temperature=0.6,
        top_k=20,
        top_p=0.97,
        early_stopping=True,
    )
    data = tokenizer.decode(final_outputs[0], skip_special_tokens=True)
    matches = re.findall(r'\[a\](.*?)\n', data, re.DOTALL)

    return data

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [56]:
sequence = "I just don't feel like myself anymore. I thought having a baby would bring me joy, but I feel empty and numb"
max_len = 50
generated_text = generate_text(model, tokenizer, sequence, max_len)

print(f"Input: {sequence}")
print(f"Generated Text: {generated_text}")

Input: I just don't feel like myself anymore. I thought having a baby would bring me joy, but I feel empty and numb
Generated Text: i just don't feel like myself anymore. i thought having a baby would bring me joy, but i feel empty and numb ) mys postpartum depression depressed aree right you see allt want to know it about something else


Generated Text: i just don't feel like myself anymore. i thought having a baby would bring me joy, but i feel empty and numb ) mys postpartum depression depressed aree right you see allt want to know it about something else


In [57]:
sequence = "How do I overcome stress?"
max_len = 50
generated_text = generate_text(model, tokenizer, sequence, max_len)

print(f"Input: {sequence}")
print(f"Generated Text: {generated_text}")

Input: How do I overcome stress?
Generated Text: how do i overcome stress?. so anxious because my exams are approaching after due a postpartum depression mood andomi insomniation as can be manage to maintain healthy sleep of usually mom mothersst ive want myself prepared for


Generated Text: how do i overcome stress?. so anxious because my exams are approaching after due a postpartum depression mood andomi insomniation as can be manage to maintain healthy sleep of usually mom mothersst ive want myself prepared for


In [58]:
sequence = "I'm irritable and angry all the time."
max_len = 50
generated_text = generate_text(model, tokenizer, sequence, max_len)

print(f"Input: {sequence}")
print(f"Generated Text: {generated_text}")

Input: I'm irritable and angry all the time.
Generated Text: i'm irritable and angry all the time., ok okay sure you are so stressed out because my depressed sleep is challenging difficult for me postpartum depression of cant control myself after childbirth a ive right soundss what


Generated Text: i'm irritable and angry all the time., ok okay sure you are so stressed out because my depressed sleep is challenging difficult for me postpartum depression of cant control myself after childbirth a ive right soundss what
